In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started With MCP Toolbox

This guide demonstrates how to quickly run
[Toolbox](https://github.com/googleapis/genai-toolbox) end-to-end in Google
Colab using Python, BigQuery, and either [Google
GenAI](https://pypi.org/project/google-genai/), [ADK](https://google.github.io/adk-docs/),
[Langgraph](https://www.langchain.com/langgraph)
or [LlamaIndex](https://www.llamaindex.ai/).

Within this Colab environment, you'll
- Set up a `BigQuery Dataset`.
- Launch a Toolbox server.
- Connect to Toolbox and develop a sample `Hotel Booking` application.

## Step 1: Set up your dataset

In this section, we will
1. Create a dataset in your bigquery project.
1. Insert example data into the dataset.

In [1]:
# @markdown Please fill in the value below and then run the cell.
BIGQUERY_PROJECT = "genaipla-aiplayground2-sa-bf14" # @param {type:"string"}
DATASET = "fullfil" # @param {type:"string"}
TABLE_ID = "order_line_life_cycle" # @param {type:"string"}

> You need to authenticate as an IAM user so this notebook can access your Google Cloud Project. This access is necessary to use Google's LLM models.

In [2]:
from google.colab import auth

# Authenticate the user for Google Cloud access
auth.authenticate_user()

In [3]:
# Create the dataset if it does not exist
from google.cloud import bigquery
from google.cloud import exceptions

bqclient = bigquery.Client(project=BIGQUERY_PROJECT)
dataset_ref = bqclient.dataset(DATASET)



## Step 2: Install and configure Toolbox

In this section, we will
1. Download the latest version of the toolbox binary.
2. Create a toolbox config file.
3. Start a toolbox server using the config file.



Download the [latest](https://github.com/googleapis/genai-toolbox/releases) version of Toolbox as a binary.

In [4]:
version = "0.16.0" # x-release-please-version
! curl -O https://storage.googleapis.com/genai-toolbox/v{version}/linux/amd64/toolbox

# Make the binary executable
! chmod +x toolbox

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  133M  100  133M    0     0   103M      0  0:00:01  0:00:01 --:--:--  103M


In [5]:
TOOLBOX_BINARY_PATH = "/content/toolbox"
SERVER_PORT = 5000

Create a tools file with the following functions:

- `Database Connection (sources)`: `Includes details for connecting to our hotels database.`
- `Tool Definitions (tools)`: `Defines five tools for database interaction:`
  - `search-hotels-by-name`
  - `search-hotels-by-location`
  - `book-hotel`
  - `update-hotel`
  - `cancel-hotel`

Our application will leverage these tools to interact with the hotels table.

For detailed configuration options, please refer to the [Toolbox documentation](https://googleapis.github.io/genai-toolbox/getting-started/configure/).



In [6]:
# Create a tools file at runtime.
# You can also upload a tools file and use that to run toolbox.
tools_file_name = "tools.yml"
file_content = f"""
sources:
  my-bigquery-source:
    kind: bigquery
    project: genaipla-aiplayground2-sa-bf14
    location: us
tools:
  search-order:
    kind: bigquery-sql
    source: my-bigquery-source
    description: Use this tool to get all details for a specific order by using its order number.
    parameters:
      - name: order_number
        type: string
        description: The unique number for the order you need to find.
    statement: SELECT * FROM `fullfil.order_line_life_cycle` WHERE order_number = CAST(@order_number AS INT64);
toolsets:
  my-bigquery-toolset:
    - search-order
"""

In [7]:
with open(tools_file_name, 'w', encoding='utf-8') as f:
    f.write(file_content)

In [8]:
TOOLS_FILE_PATH = f"/content/{tools_file_name}"

In [9]:
# Start a toolbox server
! nohup {TOOLBOX_BINARY_PATH} --tools-file {TOOLS_FILE_PATH} -p {SERVER_PORT} > toolbox.log 2>&1 &

In [10]:
# Check if toolbox is running
!sudo lsof -i :{SERVER_PORT}

COMMAND  PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
toolbox 1341 root    3u  IPv4  54172      0t0  TCP localhost:5000 (LISTEN)


## Step 3: Connect your agent to Toolbox

In this section, you will
1. Establish a connection to the tools by creating a Toolbox client.
2. Build an agent that leverages the tools and an LLM for Hotel Booking functionality.


In [11]:
# Configure gcloud.
!gcloud config set project {BIGQUERY_PROJECT}

Updated property [core/project].


> You can use ADK, LangGraph, or LlamaIndex to develop a Toolbox based application. Run one of the [Connect Using LangGraph](#scrollTo=pbapNMhhL33S), [Connect using LlamaIndex](#scrollTo=04iysrm_L_7v&line=1&uniqifier=1) or [Connect using ADK](#scrollTo=yA3rAiELIds5) sections below.



### Connect Using LangGraph

In [12]:
# Install the Toolbox Langchain package
!pip install toolbox-langchain --quiet
!pip install langgraph --quiet

# Install the Langchain llm package
# TODO(developer): replace this with another model if needed
! pip install langchain-google-vertexai --quiet
# ! pip install langchain-google-genai
# ! pip install langchain-anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0;

Create a LangGraph Order Agent which can Search, Book and Cancel hotels.

In [13]:
from langgraph.prebuilt import create_react_agent
# TODO(developer): replace this with another import if needed
from langchain_google_vertexai import ChatVertexAI
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_anthropic import ChatAnthropic
from langgraph.checkpoint.memory import MemorySaver

from toolbox_langchain import ToolboxClient

prompt = """
 You are Order Management Expert You are good at query on Order_number and extract data from the table.
"""

queries = [
    "find the order_number details for order_number 1042567961"
]

# Create an LLM to bind with the agent.
# TODO(developer): replace this with another model if needed
model = ChatVertexAI(model_name="gemini-2.0-flash-001", project=BIGQUERY_PROJECT)
# model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
# model = ChatAnthropic(model="claude-3-5-sonnet-20240620")

# Load the tools from the Toolbox server
client = ToolboxClient("http://127.0.0.1:5000")
tools = client.load_toolset()

# Create a Langraph agent
agent = create_react_agent(model, tools, checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "thread-1"}}
for query in queries:
    #inputs = {"messages": [("user", prompt + query)]}
    response = agent.invoke(inputs, stream_mode="values", config=config)
    print(response["messages"][-1].content)

NameError: name 'inputs' is not defined

In [14]:
from langgraph.prebuilt import create_react_agent
from langchain_google_vertexai import ChatVertexAI
from langgraph.checkpoint.memory import MemorySaver
from toolbox_langchain import ToolboxClient

# The system prompt defining the agent's persona
prompt = "You are an Order Management Expert. You are good at querying on Order_number and extracting data from the table."

queries = [
    "find the order_number details for order_number 1042567961"
]

# --- LLM and Toolbox Setup ---
BIGQUERY_PROJECT = "genaipla-aiplayground2-sa-bf14"

# CORRECTED: Updated to a newer, more available model name.
model = ChatVertexAI(
    model_name="gemini-1.5-flash-preview-0514",
    project=BIGQUERY_PROJECT,
    location="us-east1"
)

# Load the tools from the Toolbox server
client = ToolboxClient("http://127.0.0.1:5000")
tools = client.load_toolset()

print("--- Tools Loaded ---")
print(tools)
print("--------------------")

# Create a Langraph agent
agent = create_react_agent(model, tools, checkpointer=MemorySaver())

# --- Agent Invocation Loop ---
config = {"configurable": {"thread_id": "thread-1"}}
for query in queries:
    print(f"--- Processing Query: '{query}' ---")

    inputs = {"messages": [("system", prompt), ("user", query)]}

    # Using invoke to get the final answer directly
    final_response = agent.invoke(inputs, config=config)

    print("\n--- Final Answer ---")
    print(final_response["messages"][-1].content)



--- Tools Loaded ---
[ToolboxTool(name='search-order', description='Use this tool to get all details for a specific order by using its order number.\n\nArgs:\n    order_number (str): The unique number for the order you need to find.', args_schema=<class 'toolbox_core.utils.search-order'>)]
--------------------
--- Processing Query: 'find the order_number details for order_number 1042567961' ---

--- Final Answer ---
I am sorry, I cannot fulfill your request. The available tools lack the functionality to provide the details of a specific order. 



In [ ]:
# Install the Toolbox LlamaIndex package
!pip install toolbox-llamaindex --quiet

# Install the llamaindex llm package
# TODO(developer): replace this with another model if needed
! pip install llama-index-llms-google-genai --quiet
# ! pip install llama-index-llms-anthropic

Create a LlamaIndex Hotel Agent which can Search, Book and Cancel hotels.

In [ ]:
import asyncio
import os

from llama_index.core.agent.workflow import AgentWorkflow

from llama_index.core.workflow import Context

# TODO(developer): replace this with another import if needed
from llama_index.llms.google_genai import GoogleGenAI
# from llama_index.llms.anthropic import Anthropic

from toolbox_llamaindex import ToolboxClient

prompt = """
  You're a helpful hotel assistant. You handle hotel searching, booking and
  cancellations. When the user searches for a hotel, mention it's name, id,
  location and price tier. Always mention hotel ids while performing any
  searches. This is very important for any operations. For any bookings or
  cancellations, please provide the appropriate confirmation. Be sure to
  update checkin or checkout dates if mentioned by the user.
  Don't ask for confirmations from the user.
"""

queries = [
    "Find hotels in Basel with Basel in it's name.",
    "Can you book the Hilton Basel for me?",
    "Oh wait, this is too expensive. Please cancel it and book the Hyatt Regency instead.",
    "My check in dates would be from April 10, 2024 to April 19, 2024.",
]

async def run_agent():
    # Create an LLM to bind with the agent.
    # TODO(developer): replace this with another model if needed
    llm = GoogleGenAI(
        model="gemini-2.0-flash-001",
        vertexai_config={"project": BIGQUERY_PROJECT, "location": "us-central1"},
    )
    # llm = GoogleGenAI(
    #     api_key=os.getenv("GOOGLE_API_KEY"),
    #     model="gemini-2.0-flash-001",
    # )
    # llm = Anthropic(
    #   model="claude-3-7-sonnet-latest",
    #   api_key=os.getenv("ANTHROPIC_API_KEY")
    # )

    # Load the tools from the Toolbox server
    client = ToolboxClient("http://127.0.0.1:5000")
    tools = client.load_toolset()

    # Create a LlamaIndex agent
    agent = AgentWorkflow.from_tools_or_functions(
        tools,
        llm=llm,
        system_prompt=prompt,
    )

    # Run the agent
    ctx = Context(agent)
    for query in queries:
        response = await agent.run(user_msg=query, ctx=ctx)
        print(f"---- {query} ----")
        print(str(response))

await run_agent()

### Connect Using ADK

In [ ]:
!pip install -q google-adk
!pip install -q toolbox-core

In [ ]:
# Create an ADK Hotel Agent which can Search, Book and Cancel hotels.
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService
from google.genai import types
from toolbox_core import ToolboxSyncClient

import os

os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'
os.environ['GOOGLE_CLOUD_PROJECT'] = BIGQUERY_PROJECT
os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'

toolbox_client = ToolboxSyncClient("http://127.0.0.1:5000")

prompt = """
  You're a helpful hotel assistant. You handle hotel searching, booking and
  cancellations. When the user searches for a hotel, mention it's name, id,
  location and price tier. Always mention hotel ids while performing any
  searches. This is very important for any operations. For any bookings or
  cancellations, please provide the appropriate confirmation. Be sure to
  update checkin or checkout dates if mentioned by the user.
  Don't ask for confirmations from the user.
"""

root_agent = Agent(
    model='gemini-2.0-flash-001',
    name='hotel_agent',
    description='A helpful AI assistant.',
    instruction=prompt,
    tools=toolbox_client.load_toolset("my-toolset"),
)

session_service = InMemorySessionService()
artifacts_service = InMemoryArtifactService()
session = session_service.create_session(
    state={}, app_name='hotel_agent', user_id='123'
)
runner = Runner(
    app_name='hotel_agent',
    agent=root_agent,
    artifact_service=artifacts_service,
    session_service=session_service,
)

queries = [
    "Find hotels in Basel with Basel in it's name.",
    "Can you book the Hilton Basel for me?",
    "Oh wait, this is too expensive. Please cancel it and book the Hyatt Regency instead.",
    "My check in dates would be from April 10, 2024 to April 19, 2024.",
]

for query in queries:
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(session_id=session.id,
                        user_id='123', new_message=content)

    responses = (
      part.text
      for event in events
      for part in event.content.parts
      if part.text is not None
    )

    for text in responses:
      print(text)

### Observe the output

You can see that the `Hyatt Regency Basel` has been booked for the correct dates.

In [ ]:
sql_select = f"SELECT * FROM `{BIGQUERY_PROJECT}.{DATASET}.{TABLE_ID}`"
query_job = bqclient.query(sql_select)

print("\nQuery results:")
query_job.to_dataframe()

### Clean-Up
Conditionally delete BigQuery table and dataset in final session.

In [ ]:
bqclient.delete_table(table_ref, not_found_ok=True)

bqclient.get_dataset(dataset_ref)
tables_in_dataset = list(bqclient.list_tables(dataset_ref))
if not tables_in_dataset:
    bqclient.delete_dataset(dataset_ref, delete_contents=False, not_found_ok=True)
    print(f"Dataset '{DATASET}' deleted.")
else:
    print(f"Dataset '{DATASET}' is not empty. Skipping dataset deletion.")